# Comparison of Different Machine Learning Models

## Models Tried

1. RandomForestRegressor
    - Advantages: Why did we start with this? 
    - Drawbacks: Why did we pivot away?
1. K Nearest Neighbors
    - Advantages: Why was this the next choice?
    - Drawbacks: Why did we pivot away?
1. Gradient Boosting Classifier
    - Advantages: Why was this the next choice?
    - Drawbacks: Why did we pivot away?

## Random Forest Regressor

In [10]:
def random_forest_prediction():
    import pandas as pd

    ESN = pd.read_csv('../content/csv/FINALDATASET_EastSlopesNorth.csv')
    WSN = pd.read_csv('../content/csv/FINALDATASET_WestSlopesNorth.csv')
    WSS = pd.read_csv('../content/csv/FINALDATASET_WestSlopesSouth.csv')

    assert all(ESN.columns) == all(WSN.columns) and all(ESN.columns) == all(WSS.columns), \
        'All dataframes should have the same columns'
        
        # Combine dataframes

    DATA = pd.concat([ESN, WSN, WSS], axis=0, ignore_index=True)

    # Show any rows with NaN
    DATA[DATA.isna().any(axis=1)]

    # Todo: try dropping NaN columns instead
    DATA.fillna(DATA.mean(), inplace=True)

    # Todo: should we try to use these?
    DATA.drop(['Date', 'Area'], axis=1, inplace=True)

    # Try normalizing: does not help
    # for col in DATA.columns:
        # DATA[col] = (DATA[col]-DATA[col].mean())/DATA[col].std()
    # DATA=(DATA-DATA.min())/(DATA.max()-DATA.min())

    # Can uncomment to make prediction better
    DATA.drop(['Yesterday Danger Above Treeline', 'Yesterday Danger At Treeline', 'Yesterday Danger Below Treeline'], axis=1, inplace=True)

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score
    from sklearn.ensemble import RandomForestRegressor

    def make_prediction_rfr(df, y_col):
        x = df.drop(y_col, axis=1)
        y = df[y_col]

        x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

        rf_regressor = RandomForestRegressor(random_state=42)

        rf_regressor.fit(x_train, y_train)

        y_pred_train = rf_regressor.predict(x_train)
        y_pred = rf_regressor.predict(x_test)

        print('\t' + y_col)
        print('\t\tPrediction (Training) score:\t', '{0:.2f}'.format(r2_score(y_train, y_pred_train)))
        print('\t\tActual (Validation) score:\t', '{0:.2f}'.format(r2_score(y_test, y_pred)))

    # Drop columms that were determined to hurt the prediction
    DATA.drop(['Total Snowfall 24hr', 'Total Snowfall 72hr'], axis=1, inplace=True)

    above_treeline = DATA.drop(['At Treeline', 'Below Treeline'], axis=1)
    at_treeline = DATA.drop(['Above Treeline', 'Below Treeline'], axis=1)
    below_treeline = DATA.drop(['Above Treeline', 'At Treeline'], axis=1)

    print('Random Forest Regressor Scores')
    make_prediction_rfr(above_treeline, 'Above Treeline')
    make_prediction_rfr(at_treeline, 'At Treeline')
    make_prediction_rfr(below_treeline, 'Below Treeline')

random_forest_prediction()

/tmp/ipykernel_21919/2576179185.py:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  DATA.fillna(DATA.mean(), inplace=True)


Random Forest Regressor Scores
	Above Treeline
		Prediction (Training) score:	 0.86
		Actual (Validation) score:	 0.23
	At Treeline
		Prediction (Training) score:	 0.86
		Actual (Validation) score:	 0.28
	Below Treeline
		Prediction (Training) score:	 0.86
		Actual (Validation) score:	 0.15


## K Nearest Neighbors

In [11]:
import pandas as pd

ESN = pd.read_csv('../content/csv/FINALDATASET_EastSlopesNorth.csv')
WSN = pd.read_csv('../content/csv/FINALDATASET_WestSlopesNorth.csv')
WSS = pd.read_csv('../content/csv/FINALDATASET_WestSlopesSouth.csv')

assert all(ESN.columns) == all(WSN.columns) and all(ESN.columns) == all(WSS.columns), \
    'All dataframes should have the same columns'
    
    # Combine dataframes

DATA = pd.concat([ESN, WSN, WSS], axis=0, ignore_index=True)

# Show any rows with NaN
DATA[DATA.isna().any(axis=1)]

# Todo: try dropping NaN columns instead
DATA.fillna(DATA.mean(), inplace=True)

# Todo: should we try to use these?
DATA.drop(['Date', 'Area'], axis=1, inplace=True)

# Try normalizing: does not help
# for col in DATA.columns:
    # DATA[col] = (DATA[col]-DATA[col].mean())/DATA[col].std()
# DATA=(DATA-DATA.min())/(DATA.max()-DATA.min())

# Can uncomment to make prediction better
DATA.drop(['Yesterday Danger Above Treeline', \
           'Yesterday Danger At Treeline', 'Yesterday Danger Below Treeline'], \
            axis=1, inplace=True)

NORMALIZED = DATA.copy(deep=True)
for col in NORMALIZED.columns:
    if col not in (
        'Above Treeline',
        'At Treeline', 
        'Below Treeline'
        'Was High Winds 24hr', 
        'Was Heavy Snowfall 24hr'):
        NORMALIZED[col] = (NORMALIZED[col]-NORMALIZED[col].mean())/NORMALIZED[col].std()

from sklearn.model_selection import train_test_split

x = NORMALIZED.drop(['Above Treeline', 'At Treeline', 'Below Treeline'], axis=1)
y = NORMALIZED['At Treeline']

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=24) 

model.fit(x_training_data, y_training_data)

predictions = model.predict(x_test_data)

print(model.score(x_training_data, y_training_data))
print(model.score(x_test_data, y_test_data))

0.6051136363636364
0.5393258426966292


/tmp/ipykernel_21919/1578121937.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  DATA.fillna(DATA.mean(), inplace=True)
